# Hoja de Trabajo 1: Experimentación con Redes Neuronales

## Introducción
Este ejercicio tiene como objetivo profundizar en la comprensión de las redes neuronales mediante la experimentación. A través de variaciones en la arquitectura y parámetros, se busca entender cómo se comporta y aprende una red neuronal. Utilizaremos el dataset "Heart Disease" disponible en el UCI Machine Learning Repository.

## Dataset "Heart Disease"
Puedes descargar el dataset directamente desde el siguiente enlace:
http://archive.ics.uci.edu/dataset/45/heart+disease. Recuerda que vamos a estar utilizando CRISP-DM para crear los modelos. El primer paso es comprender los datos. Ese link te proporciona la información necesaria para entender el dataset. Las columnas y el rol que cada columna va a jugar en tu modelo (variables independientes y dependiente). Como menciona la documentación, utiliza solo la base de datos de Cleveland.

## Sección 1: Preparación de Datos
* Descarga y carga el dataset "Heart Disease".
* Realiza un breve análisis exploratorio: ¿Qué tipo de datos contiene? ¿Cuántos ejemplos hay?. 
* Procesa el dataset: normaliza o estandariza si es necesario, divide el dataset en conjuntos de entrenamiento y prueba.

Nuestro enfoque de hoy no es un mega modelo sino comprender mejor como se comportan las redes neuronales. Realiza lo necesario pero nada muy extravagante.

### Carga de datos y análisis general

In [101]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [73]:
col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
data =pd.DataFrame(pd.read_csv("processed.cleveland.data", sep= ',', names=col_names))
print("Información general del dataset:")
print(data.info())
print("")
print(f"La matriz tiene una dimensión de: {data.shape}")
print(f"Se tienen los tipos de datos: {data.dtypes.unique().tolist()}")

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  num       303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB
None

La matriz tiene una dimensión de: (303, 14)
Se tienen los tipos de datos: [dtype('float64'), dtype('O'), dtype('int64')]


### Distribución y estadísticas generales del dataset

In [35]:
print("Distribución y composición del dataset:")
print(data.describe())

Distribución y composición del dataset:
              age         sex          cp    trestbps        chol         fbs  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000   
mean    54.438944    0.679868    3.158416  131.689769  246.693069    0.148515   
std      9.038662    0.467299    0.960126   17.599748   51.776918    0.356198   
min     29.000000    0.000000    1.000000   94.000000  126.000000    0.000000   
25%     48.000000    0.000000    3.000000  120.000000  211.000000    0.000000   
50%     56.000000    1.000000    3.000000  130.000000  241.000000    0.000000   
75%     61.000000    1.000000    4.000000  140.000000  275.000000    0.000000   
max     77.000000    1.000000    4.000000  200.000000  564.000000    1.000000   

          restecg     thalach       exang     oldpeak       slope         num  
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000  
mean     0.990099  149.607261    0.326733    1.039604    1.600660    0

In [36]:
print("Primeras líneas:")
print(data.head())

Primeras líneas:
    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca thal  num  
0    3.0  0.0  6.0    0  
1    2.0  3.0  3.0    2  
2    2.0  2.0  7.0    1  
3    3.0  0.0  3.0    0  
4    1.0  0.0  3.0    0  


### Tipo de variables

In [76]:
def getDateColTypes (df):
    categoricas=[]
    continuas=[]
    discretas=[]

    for colName in df.columns:
        if(df[colName].dtype=='object'):
            categoricas.append(colName)
        else:
            if((df[colName].dtype=='int64')or (df[colName].dtype=='float64')or (df[colName].dtype=='uint8')):
                if (len(df[colName].unique())<=30):
                    discretas.append(colName)
                else:
                    continuas.append(colName)
    return categoricas, continuas, discretas

categoricas, continuas, discretas=getDateColTypes(data)

tipos_var = {"Categoricas":categoricas, "Contínuas": continuas, "Discretas":discretas}

In [77]:
tipos_var

{'Categoricas': ['ca', 'thal'],
 'Contínuas': ['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],
 'Discretas': ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'num']}

In [72]:
for i, lista in tipos_var.items():
    tipos = ", ".join(map(str, lista)) if lista else "No hay"
    print(f"Las variables {i}: {tipos}")

Las variables Categoricas: ca, thal
Las variables Contínuas: age, trestbps, chol, thalach, oldpeak
Las variables Discretas: sex, cp, fbs, restecg, exang, slope, num


### Validación variables con nulos

In [78]:
cols_Nan=[col for col in data.columns if(data[col].isnull().mean()>0)]
cols_Nan

[]

Debido a que no se tienen variables con nulos, se procede a realizar análisis de las variables imputadas con el caracter ?

Se convierten las variables a numéricos y se determina la cantidad de datos faltantes para determinar su tratamiento

In [100]:
# Se copian las columnas como NaN 
data["ca_na"]= pd.to_numeric(data['ca'], errors='coerce')
data["thal_na"]= pd.to_numeric(data['thal'], errors='coerce')

# Se obtiene el porcentaje de datos con NaN
media=round(data['ca_na'].isnull().mean()*100,2)
print(f'Para la variable ca_na, se tiene {media = } % valores nulos')
media=round(data['thal_na'].isnull().mean()*100,2)
print(f'Para la variable thal_na, se tiene {media = } % valores nulos')

Para la variable ca_na, se tiene media = 1.32 % valores nulos
Para la variable thal_na, se tiene media = 0.66 % valores nulos


In [109]:
print(data[['ca_na','thal_na']].describe())
media=round(data['ca_na'].mean(),0)
mediana=round(data['ca_na'].median(),0)

            ca_na     thal_na
count  299.000000  301.000000
mean     0.672241    4.734219
std      0.937438    1.939706
min      0.000000    3.000000
25%      0.000000    3.000000
50%      0.000000    3.000000
75%      1.000000    7.000000
max      3.000000    7.000000


In [ ]:
# Representación gráfica
fig=plt.figure()
ax=fig.add_subplot(111)

data['ca'].plot.density(color='red',label='Original')
data['ca_Mean'].plot.density(color='blue',label='Media')
data['ca_Median'].plot.density(color='green',label='Mediana')
ax.legend()
plt.show()


## Sección 2: Construcción de la Red Neuronal

Construye una red neuronal básica utilizando Keras.

* Define la arquitectura: número de capas, neuronas por capa, función de activación.
* Compila el modelo especificando la función de pérdida, el optimizador y métricas adicionales.
* Realiza una breve descripción escrita de la arquitectura que has definido.

In [17]:
### Tu código aquí

In [18]:
### Tu descripción aquí

## Sección 3: Experimentación

Utiliza ciclos para las siguientes pruebas. Debes entrenar varios modelos variando el parámetro indicado en cada inciso según el rango y paso proporcionado. Recuerda almacenar tus resultados porque te servirán en la siguiente sección.

### Variabilidad en Capas Ocultas:

Las redes con demasiadas capas pueden tardar mucho tiempo en entrenarse, especialmente si no se dispone de un hardware adecuado.

* Rango: Entre 1 y 10 capas.
* Paso: 1 capa.
* Nota: En muchos problemas, incluso 2-3 capas son suficientes para obtener buenos resultados. Por encima de eso, a veces se necesita una arquitectura especializada o un problema muy complejo para justificar más capas.

In [19]:
### Tu código aquí

### Variabilidad en Neuronas:

El número de neuronas por capa influye en la capacidad de la red. Sin embargo, demasiadas neuronas pueden hacer que el entrenamiento sea más lento y puede causar sobreajuste.

* Rango: Entre 10 y 1000 neuronas.
* Paso: Incremento logarítmico

In [20]:
### Tu código aquí

### Variabilidad en Épocas:

Demasiadas épocas sin una técnica de parada temprana pueden llevar al sobreajuste.

* Rango: Entre 10 y 5000 épocas.
* Paso: Incremento logarítmico

In [21]:
### Tu código aquí

## Tamaño de Batch:

El tamaño del batch influye en la velocidad de entrenamiento y la estabilidad del modelo. Un batch muy grande puede no caber en la memoria, mientras que uno muy pequeño puede hacer que el entrenamiento sea inestable.

* Rango: Entre 1 (Stochastic Gradient Descent) y el tamaño del conjunto de datos (Batch Gradient Descent).
* Paso: Incremento logarítmico

In [22]:
### Tu código aquí

## Sección 4: Análisis y Reflexión

Con los datos recopilados de las pruebas anteriores, genera gráficos que permitan comparar y analizar el rendimiento del modelo. Te recomendamos utilizar librerías como matplotlib o seaborn para esta tarea. Luego de cada gráfico debes incluir un conclusión de lo que has aprendido con la experimentación y los gráficos.

### Variabilidad en Capas Ocultas

Realiza 2 gráficos.  Uno con cada una de las variables indicadas para el eje Y.

* Eje X: Número de capas ocultas.
* Eje Y: Precisión y pérdida.

In [23]:
### Tu código aquí

In [24]:
### Tu conclusión aquí

### Como definiste la cantidad de neuronas para tus capas ocultas?

### Variabilidad en Neuronas

Realiza 2 gráficos.  Uno con cada una de las variables indicadas para el eje Y.

* Eje X: Número de neuronas.
* Eje Y: Precisión y pérdida.

In [25]:
### Tu código aquí

In [26]:
### Tu conclusión aquí

### Cosideras necesario cambiar la cantidad de neuronas para el ejercicio sobre variabilidad en capas ocultas?

### Variabilidad en Épocas

Realiza 1 gráfico

* Eje X: Número de épocas.
* Eje Y: Precisión y pérdida.

In [27]:
### Tu código aquí

In [28]:
### Tu conclusión aquí

### Tamaño de Batch:

Realiza 2 gráficos.  Uno con cada una de las variables indicadas para el eje Y.
* Eje X: Tamaño del batch.
* Eje Y: Precisión y pérdida.

In [29]:
### Tu código aquí

In [30]:
### Tu conclusión aquí